# Problem Statement

To develop multiclass classification algorithm to predict the drug type based on the given features

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import confusion_matrix,classification_report, roc_auc_score,matthews_corrcoef,precision_score, recall_score, f1_score, accuracy_score


import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('../input/drug-classification/drug200.csv')

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df['BP'].value_counts()

In [ ]:
df['Cholesterol'].value_counts()

In [ ]:
labelencoder = LabelEncoder()

In [ ]:
def label_encode(cat_feature):
    return labelencoder.fit_transform(df[cat_feature])

In [ ]:
cat_features = ['Sex','BP','Cholesterol','Drug']

In [ ]:
for feature in cat_features:
    df[feature+'_cat'] = label_encode(feature)

In [ ]:
df.head()

In [ ]:
X,y = df[['Age','Sex_cat','BP_cat','Cholesterol_cat','Na_to_K']],df['Drug_cat']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,stratify=y,random_state=42)

In [ ]:
xgb_classifier = xgb.XGBClassifier()

In [ ]:
xgb_classifier.fit(X_train,y_train)

In [ ]:
 prediction = xgb_classifier.predict(X_test)

In [ ]:
print(classification_report(prediction,y_test))

In [ ]:
print("Precision:{}".format(precision_score(prediction,y_test,average='weighted')))
print("Recall:{}".format(recall_score(prediction,y_test,average='weighted')))
print("F1 Score:{}".format((f1_score(prediction,y_test,average='weighted'))))

# Hyperparamete tuning


Reference: https://proceedings.neurips.cc/paper/2012/file/05311655a15b75fab86956663e1819cd-Paper.pdf

Machine learning models have parameters and hyperparameters. Parameters are learnt from the data and hyperparameters are pre-defined for a model. For example, in linear/logistic regression, the coefficients and biases are the parameters that is being optimised while training, whereas learning rate is one hyperparameter that is set to a constant before training the model. The process of finding the best hyperparameters from a range of values is called hyperparameter tuning.

## Bayesian optimization for hyperparameter tuning


Bayesian optimization (BO) is an automated procedure for hyperparameter tuning. BO uses gaussian process to model mean and variance of the target function.

### Gaussian process

Guassian process is a prior distribution on functions, and in the case of hyperparameter tuning it will be of the form:

$
\begin{align}
f:\chi \rightarrow {\mathbb{R}} 
\end{align}
$ <br>
<br>
Where $\chi$ is the space of hyperparameter set we are tuning, and $f$ is the target function decided based on the criteria on which the hyperparameters needs to be tuned. For a gaussian process, $p(f|\chi)$ follows a normal distribution. In our problem, we define the mean of weighted F1 score from five fold cross validation as the target function. 

### Acquisition function

It is assumed that the function $f(x)$ is drawn from a gaussain process prior and our observation is of the form ${x_i,y_i}$, where <br>

$ y_i \thicksim N(f(x_i), ν) $ , $ν$ is the variance of noise introduced into the function observations.

This prior and the data induce a posterior over functions,the acquisition function, $a:\chi \rightarrow {\mathbb{R^{+}}} 
$, determines what points in $\chi$ to be evaluated next.

There are different choices for acquisition functions, mainly <br>

1) Upper Confidence Bounds method <br>
2) Expected improvement method <br>
3) Probability Of Improvement method <br>


Acquisiton function follows a greedy approach to find the optimum hyperparameters in each step, and with a fixed step of iteration, we are  expected to get the suitable hyperparameters for our model.






For understanding how bayesian optimization works,we will finetune for 'max_depth','gamma','n_estimators','n_estimators' and 'subsample' hyperparameters.

In [ ]:
from bayes_opt import BayesianOptimization

In [ ]:

def bo_params_xgb(max_depth, gamma,learning_rate,n_estimators,subsample):
    
    params = {
        'max_depth': int(max_depth),
        'gamma': gamma,
        'learning_rate':learning_rate,
        'subsample': subsample,
        'eta': 0.1,
        'eval_metric': 'auc',
        'n_estimators':int(n_estimators)
    }
    
    scores = cross_val_score(xgb.XGBClassifier(random_state=123, **params,use_label_encoder=False),
                             X_train, y_train,cv=5,scoring="f1_weighted").mean()
    
    #cv_result = xgb.cv(params, train_dmatrix, nfold=5)
    return scores.mean()

In [ ]:
xgb_bo = BayesianOptimization(bo_params_xgb, {'max_depth': (3, 10),
                                             'gamma': (0, 1),
                                             'learning_rate':(0,1),
                                              'subsample':(0.5,1),
                                              'n_estimators':(100,200)
                                             })

In [ ]:
results = xgb_bo.maximize(n_iter=200, init_points=2)

In [ ]:
params = xgb_bo.max['params']
print(params)

In [ ]:
params['max_depth']= int(params['max_depth'])
params['n_estimators']= int(params['n_estimators'])

In [ ]:
xgb_classifier2 = xgb.XGBClassifier(**params,use_label_encoder=False)
xgb_classifier2.fit(X_train,y_train)

In [ ]:
 prediction = xgb_classifier2.predict(X_test)

In [ ]:
print(classification_report(prediction,y_test))

In [ ]:
print("Precision:{}".format(precision_score(prediction,y_test,average='weighted')))
print("Recall:{}".format(recall_score(prediction,y_test,average='weighted')))
print("F1 Score:{}".format((f1_score(prediction,y_test,average='weighted'))))

# WIP

1) Detailed mathematical formulation of each acquistion functions <br>
2) K-fold cross validation for model validation